In [1]:
import requests
import env
from requests.models import Request, Response
from env import Client_ID
import pandas as pd
from igdb.igdbapi_pb2 import GameResult
from typing import Dict, List, Optional, Union, cast

In [2]:
def get_api():   
url = env.get_db_url()
response = requests.post(url)
data = response.json()
access_token = data['access_token']
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *;'
r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)
    return access_token, r

In [ ]:
r

In [ ]:
access_token, r = get_api()

In [3]:
from igdb.wrapper import IGDBWrapper
wrapper = IGDBWrapper(f'{Client_ID}', f'Bearer {access_token}')

In [4]:
testgames = []
for i in range(0, 409):
    games = wrapper.api_request('games', 'fields *; limit 500;' f'offset {i * 409};')
    testgames.append(games)

HTTPError: 401 Client Error: Unauthorized for url: https://api.igdb.com/v4/games

In [ ]:
len(testgames)

In [ ]:
# Opening JSON file
f = open('data.json')

# returns JSON object as
# a dictionary
data1 = json.load(f)

#converting it into dataframe
df = pd.read_json(data1, orient ='index')

In [ ]:
import json
games = pd.DataFrame(r.json())

In [ ]:
df = pd.DataFrame(eval(games_message))

In [ ]:
games.shape